In [1]:
import pandas as pd

from utils import *
from preprocessing import *
from google_utils import *

{"--IMMITATION--": "I'm beaming with joy!"}


In [2]:
df_list_perspective = read_df_list('combat_df_list_imms_1_full_perspective.pkl')
check_column_type(df_list_perspective[0][0])


text <class 'str'>
imm_1 <class 'str'>
imm_1_check <class 'numpy.bool_'>
perspective <class 'dict'>


In [3]:
# 对于每个df_list_perspective包含的list, 遍历其中的dataframe, 检查其每一行的perspective列的数据类型, 保存到新的一列
for df_list in df_list_perspective:
    for df in df_list:
        df['perspective_type'] = df['perspective'].apply(lambda x: type(x))



In [4]:
# 找出所有的perspective列的数据类型
perspective_type_set = set()
for df_list in df_list_perspective:
    for df in df_list:
        perspective_type_set.update(set(df['perspective_type']))
        
perspective_type_set

{NoneType, dict}

In [7]:
# 找出所有包含NoneType的df

for ind1, df_list in enumerate(df_list_perspective):
    for ind2, df in enumerate(df_list):
        if type(None) in set(df['perspective_type']):
            print(ind1, ind2)
            print(df['text'].values)

90 1
['&gt; Uh, no. This isn\'t true in the slightest. They have very different pharmacology. Caffeine\'s liability to lead to serious addiction and physical dependence is tiny compared to cocaine.\n\nI\'m going to stop you there. Cocaine isn\'t physically addictive. Even a cursory glance at a Wikipedia article would have told you that. I\'m not sure how you wrote so much just now about a topic you know so little about. That means, of course, that caffeine, which can easily result in physical dependence, has a far greater "liability" for it than cocaine.'
 '"Cocaine is addictive due to its effect on the reward pathway in the brain." \n- [A cursory glance at the wiki article](https://en.wikipedia.org/wiki/Cocaine)'
 "Cool, where did I say cocaine isn't addictive? Cocaine is psychologically addictive just like how delicious food, comfort, etc. are. It is *not* physically addictive."]
90 2
["The counter-cultures that used/abused those drugs may have been what brought the drugs to the atte

In [ ]:
# 每遇到一个包含NoneType的df, 就对其进行处理

# 找出符合条件的dataframe里包含NoneType的行, 对其'text'列进行处理


In [14]:
def handle_dataframe(df):
    for ind, row in df.iterrows():
        if type(row['perspective']) == type(None):
            print('text'+"\n"+row['text'])
            row['perspective']=get_score_backoff(row['text'])
    return df

In [15]:
for ind1, df_list in enumerate(df_list_perspective):
    print(ind1)
    for ind2, df in enumerate(df_list):
        if type(None) in set(df['perspective_type']):
            handle_dataframe(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
text
"Cocaine is addictive due to its effect on the reward pathway in the brain." 
- [A cursory glance at the wiki article](https://en.wikipedia.org/wiki/Cocaine)
<HttpError 400 when requesting https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze?key=XXXXX&alt=json returned "Attribute REASONING_EXPERIMENTAL does not support request languages: om". Details: "[{'@type': 'type.googleapis.com/google.commentanalyzer.v1alpha1.Error', 'errorType': 'LANGUAGE_NOT_SUPPORTED_BY_ATTRIBUTE', 'languageNotSupportedByAttributeError': {'detectedLanguages': ['om'], 'attribute': 'REASONING_EXPERIMENTAL'}}]">
<HttpError 400 when requesting https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze?key=XXXXX&alt=json returned "Att

In [43]:
fixed_keys = [
    'AFFINITY_EXPERIMENTAL', 'COMPASSION_EXPERIMENTAL', 'CURIOSITY_EXPERIMENTAL',
    'IDENTITY_ATTACK', 'IDENTITY_ATTACK_EXPERIMENTAL', 'INSULT',
    'INSULT_EXPERIMENTAL', 'NUANCE_EXPERIMENTAL', 'PERSONAL_STORY_EXPERIMENTAL',
    'PROFANITY', 'PROFANITY_EXPERIMENTAL', 'REASONING_EXPERIMENTAL',
    'RESPECT_EXPERIMENTAL', 'SEVERE_TOXICITY', 'SEVERE_TOXICITY_EXPERIMENTAL',
    'SEXUALLY_EXPLICIT', 'THREAT', 'THREAT_EXPERIMENTAL',
    'TOXICITY', 'TOXICITY_EXPERIMENTAL'
]


def convert(x, fixed_keys=fixed_keys):
    if type(x) == type(None):
        return None
    else:
        x= [x[key] for key in fixed_keys]
        return [x[0]['spanScores'][0]['end']] + [x[i]['summaryScore']['value'] for i in range(0, 20)]
def format_perspective_dict_into_list(df):
    df['perspective_ls'] = df['perspective'].apply(convert)

In [44]:
for ind1, df_list in enumerate(df_list_perspective):
    for ind2, df in enumerate(df_list):
        format_perspective_dict_into_list(df)
        
# save to pickle
save_df_list(df_list_perspective, 'combat_df_list_imms_1_full_perspective_list.pkl')

In [ ]:
# 使用'perspective_ls'作为X, 预测Y即imm_1_check
# 使用随机森林

In [45]:
flattened =pd.concat([df for ls in df_list_perspective for df in ls])
sort = flattened[flattened['perspective_ls'].apply(lambda x: isinstance(x, list))]
print(sort.shape, sort.columns)

(34805, 6) Index(['text', 'imm_1', 'imm_1_check', 'perspective', 'perspective_type',
       'perspective_ls'],
      dtype='object')


In [46]:
# 确认sort的['perspective_ls']列的数据类型和长度
print(sort['perspective_ls'].apply(type).value_counts())
print(sort['perspective_ls'].apply(len).value_counts())


perspective_ls
<class 'list'>    34805
Name: count, dtype: int64
perspective_ls
21    34805
Name: count, dtype: int64


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV

# vectors = []
# targets = []
# 
# for index, row in sort.iterrows():
#     vectors.append([row['perspective_ls'][0]['spanScores'][0]['end']]
#                    + [row['perspective_ls'][i]['summaryScore']['value'] for i in range(0, 20)])
#     targets.append(row['imm_1_check'])
#         
# X=np.array(vectors)
# Y=np.array(targets)
# 假设 'perspective_ls' 是特征列，'imm_1_check' 是目标列
X = np.array(sort['perspective_ls'].tolist())  # 如果 'perspective_ls' 是嵌套列表
Y = sort['imm_1_check']  # 将你的目标数据填入Y
print(X[0])
print(X.shape, Y.shape)
# 分割数据为训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 如果你的 X_train 和 X_test 不需要重新 reshape，不要再进行下列的 reshape 操作
# 如果需要变换形状，请直接使用 reshape
# 例如，如果 X_train 是二维数组，可以使用以下操作
# X_train = X_train.reshape(-1, 1)
# X_test = X_test.reshape(-1, 1)

# 创建并训练随机森林模型
model = RandomForestClassifier(random_state=42)
model.fit(X_train, Y_train)

# 预测
Y_pred = model.predict(X_test)

# 评估模型
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Model accuracy: {accuracy:.2f}")



[8.00000000e+01 1.02570616e-01 8.67879200e-02 8.16133540e-02
 2.58060360e-02 2.58060360e-02 1.77431960e-02 1.77431960e-02
 2.10842300e-01 9.59158600e-02 2.13301570e-02 2.13301570e-02
 3.83785370e-01 4.93806960e-01 1.96456900e-03 1.96456900e-03
 2.11138260e-02 6.59569070e-03 6.59569070e-03 6.30274340e-02
 6.30274340e-02]
(34805, 21) (34805,)
Model accuracy: 0.95


In [48]:
fixed_keys = ['len',
    'AFFINITY_EXPERIMENTAL', 'COMPASSION_EXPERIMENTAL', 'CURIOSITY_EXPERIMENTAL',
    'IDENTITY_ATTACK', 'IDENTITY_ATTACK_EXPERIMENTAL', 'INSULT',
    'INSULT_EXPERIMENTAL', 'NUANCE_EXPERIMENTAL', 'PERSONAL_STORY_EXPERIMENTAL',
    'PROFANITY', 'PROFANITY_EXPERIMENTAL', 'REASONING_EXPERIMENTAL',
    'RESPECT_EXPERIMENTAL', 'SEVERE_TOXICITY', 'SEVERE_TOXICITY_EXPERIMENTAL',
    'SEXUALLY_EXPLICIT', 'THREAT', 'THREAT_EXPERIMENTAL',
    'TOXICITY', 'TOXICITY_EXPERIMENTAL'
]

feature_importances = model.feature_importances_
for name, importance in sorted(zip(fixed_keys, feature_importances[1:]), key=lambda x: x[1], reverse=True):
    print(f"Feature: {name}, Importance: {importance:.4f}")
    
import matplotlib.pyplot as plt
import seaborn as sns
# 绘制特征重要性图
plt.figure(figsize=(10, 8))
sns.barplot(x=feature_importances[1:], y=fixed_keys)
plt.title("Feature Importances")
plt.show()

Feature: SEVERE_TOXICITY_EXPERIMENTAL, Importance: 0.0885
Feature: SEVERE_TOXICITY, Importance: 0.0572
Feature: RESPECT_EXPERIMENTAL, Importance: 0.0558
Feature: SEXUALLY_EXPLICIT, Importance: 0.0542
Feature: REASONING_EXPERIMENTAL, Importance: 0.0479
Feature: THREAT_EXPERIMENTAL, Importance: 0.0474
Feature: CURIOSITY_EXPERIMENTAL, Importance: 0.0474
Feature: THREAT, Importance: 0.0471
Feature: IDENTITY_ATTACK, Importance: 0.0467
Feature: TOXICITY, Importance: 0.0465
Feature: INSULT_EXPERIMENTAL, Importance: 0.0451
Feature: COMPASSION_EXPERIMENTAL, Importance: 0.0442
Feature: PROFANITY_EXPERIMENTAL, Importance: 0.0431
Feature: PERSONAL_STORY_EXPERIMENTAL, Importance: 0.0428
Feature: PROFANITY, Importance: 0.0417
Feature: AFFINITY_EXPERIMENTAL, Importance: 0.0413
Feature: len, Importance: 0.0391
Feature: NUANCE_EXPERIMENTAL, Importance: 0.0386
Feature: INSULT, Importance: 0.0379
Feature: IDENTITY_ATTACK_EXPERIMENTAL, Importance: 0.0363


ValueError: All arrays must be of the same length

<Figure size 1000x800 with 0 Axes>

In [40]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
print("Classification Report:")
print(classification_report(Y_test, Y_pred))

Classification Report:
              precision    recall  f1-score   support

       False       0.68      0.26      0.38       427
        True       0.95      0.99      0.97      6534

    accuracy                           0.95      6961
   macro avg       0.82      0.63      0.67      6961
weighted avg       0.94      0.95      0.94      6961



In [ ]:
vectors = []
targets = []

for df in sort:
    for index, row in df.iterrows():
        vectors.append([row['perspective_ls'][0]['spanScores'][0]['end']]
                       + [row['perspective_ls'][i]['summaryScore']['value'] for i in range(0, 20)])
        targets.append(row['imm_1_check'])
        
